In [1]:
import json
# Assignment - 3  NLP

This has some great tips as always and is helping me to complete my Good Eats collection.  I haven't tried any of the recipes yet, but I will soon.  Sometimes it's just lovely to let Alton entertain us.
This is a great pastry guide.  I love how Alton's collections can break it down so baking isn't so mystical and scary.  I might even try some of these recipes some day.


In [7]:
import json
import pandas as pd
df = pd.read_json('./sampled_data/movie_reviews.json',lines=True)
#f = json.loads(open('./sampled_data/movie_reviews.json','r').read())


"This has some great tips as always and is helping me to complete my Good Eats collection.  I haven't tried any of the recipes yet, but I will soon.  Sometimes it's just lovely to let Alton entertain us."

In [8]:
def save_to_text_file(file):
    with open(file,'w') as f:
        # save the column reviewText to a text file
        for line in df['reviewText'].to_list():
            f.write(line+"\n")


In [9]:
save_to_text_file('./sampled_data/corpus.txt')

In [2]:
import torchtext
from torchtext.data.utils import get_tokenizer
import re
from cleantext import clean

In [8]:
# Preprocess the data 
tokenizer = get_tokenizer('basic_english')

def replace_dates(text):
        date_format_a = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' <DATE> ', text)
        date_format_b = re.sub(
            r'[A-Za-z]{2,8}\s\d{1,2},?\s\d {4}', ' <DATE> ', date_format_a)
        date_format_c = re.sub(
            r'\d{2} [A-Z][a-z]{2,8} \d{4}', ' <DATE> ', date_format_b)
        return date_format_c

def replace_hash_tags(text):
        return re.sub(r'(\s|^)#(\w+)', ' <HASHTAG> ', text)

def remove_special_characters(text):
        # remove all special characters 
        return re.sub(r'[^A-Za-z0-9\s]', ' ', text)

def remove_extra_spaces(text):
        return re.sub(r'\s{2,}', ' ', text)

def replace_hyphenated_words(text):
        # replace hyphenated words with words seperated by space
        return re.sub(r'(\w+)-(\w+)', r' \1 \2 ', text)

def read_data(filename,line_count,atleast=10):
    with open(filename, 'r') as f:
        lines = []
        ctr = 0
        for line in f.readlines():
            line = line.strip()
            line = re.sub(r'<|>', ' ', line)
            line = replace_dates(line)
            line = replace_hyphenated_words(line)
            line = replace_hash_tags(line)
            # remove < and > from the text
            line = clean(line, no_emoji=True,
                         no_urls=True,
                         no_emails=True,
                         no_phone_numbers=True,
                         no_currency_symbols=True,           
                         replace_with_url=" <URL> ",
                         replace_with_email=" <EMAIL> ",
                         replace_with_phone_number=" <PHONE> ",
                         replace_with_currency_symbol=" <CURRENCY> ",
                         lower=True)
            line = remove_special_characters(line)
            line = clean(line,no_numbers=True,no_digits=True,no_punct=True, replace_with_number=" <NUMBER> ",replace_with_digit=" ",replace_with_punct="")
            line = remove_extra_spaces(line)
            tokens=tokenizer(line)
            if len(tokens)>atleast:
                lines.append(tokens)
                ctr+=1
            if ctr >= line_count:
                break
    return lines


def save_data(filename, lines):
    # Save the data to a file
    with open(filename, 'w')as f:
        for line in lines:
            line = ' '.join(line)
            f.write(line.strip()+'\n')


In [20]:
def create_sampled_dataset(sentences,input_file,line_count,atleast=10):
    lines = []
    ctr= 0
    with open(input_file, 'r') as f:
        for _ in range(sentences):
            line=json.loads(f.readline().strip())['reviewText']
            line = line.strip()
            line = re.sub(r'<|>', ' ', line)
            line = replace_dates(line)
            line = replace_hyphenated_words(line)
            line = replace_hash_tags(line)
            # remove < and > from the text
            line = clean(line, no_emoji=True,
                        no_urls=True,
                        no_emails=True,
                        no_phone_numbers=True,
                        no_currency_symbols=True,           
                        replace_with_url=" <URL> ",
                        replace_with_email=" <EMAIL> ",
                        replace_with_phone_number=" <PHONE> ",
                        replace_with_currency_symbol=" <CURRENCY> ",
                        lower=True)
            line = remove_special_characters(line)
            line = clean(line,no_numbers=True,no_digits=True,no_punct=True, replace_with_number=" <NUMBER> ",replace_with_digit=" ",replace_with_punct="")
            line = remove_extra_spaces(line)
            tokens=tokenizer(line)
            if len(tokens)>atleast:
                lines.append(tokens)
                ctr+=1
            if ctr >= line_count:
                break
    return lines
                # f1.write(f.readline())



In [21]:
lines = create_sampled_dataset(2,'./data/reviews_Movies_and_TV.json',80000,10)
save_data('./data/processed_data/corpus_cleaned.txt', lines)

In [4]:
def get_vocab(file_path):
    vocab = set()
    with open(file_path, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            tokens = line.split()
            vocab.update(tokens)
    vocab=sorted(vocab)
    return vocab

In [2]:
vocab = get_vocab('./processed_data/corpus_cleaned.txt')


In [4]:
import json
json.dump(vocab,open('./processed_data/vocab.json','w'))